### ----SETUP----

In [10]:
pip install pymongo

   ---------------------------------------- 0.0/910.9 kB ? eta -:--:--
   ---------------------------------------- 910.9/910.9 kB 6.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
from pymongo import MongoClient

In [4]:
from datetime import datetime

Connect to local MongoDB instance

In [7]:
client = MongoClient("mongodb://localhost:27017/")

Create database and collection

In [10]:
db = client["collab_editor"]
documents_col = db["documents"]

### ----INSERT DUMMY DATA----

Dummy documents

In [14]:
docs = [
    {
        "_id": "doc_1",
        "title": "Research Notes",
        "created_at": datetime(2025, 11, 8, 12, 0),
        "owner": {"user_id": "u1", "name": "Kostas", "email": "kostas@example.com"},
        "collaborators": [
            {"user_id": "u2", "name": "Elena", "role": "editor"},
            {"user_id": "u3", "name": "George", "role": "viewer"}
        ],
        "edits": [
            {
                "edit_id": "e1",
                "user_id": "u2",
                "timestamp": datetime(2025, 11, 8, 13, 15),
                "content_added": "Introduction paragraph added"
            },
            {
                "edit_id": "e2",
                "user_id": "u3",
                "timestamp": datetime(2025, 11, 8, 14, 0),
                "content_added": "Added references section"
            }
        ]
    },
    {
        "_id": "doc_2",
        "title": "Team Meeting Notes",
        "created_at": datetime(2025, 11, 9, 9, 30),
        "owner": {"user_id": "u2", "name": "Elena", "email": "elena@example.com"},
        "collaborators": [
            {"user_id": "u1", "name": "Kostas", "role": "editor"}
        ],
        "edits": []
    }
]
documents_col.insert_many(docs)
print("Inserted sample documents successfully!")


Inserted sample documents successfully!


### ----EXAMPLE QUERIES----

a) Get a user's data (documents owned by user u1)

In [18]:
print("\nDocuments owned by user u1:")
for doc in documents_col.find({"owner.user_id": "u1"}):
    print(doc["title"], "->", doc["owner"])


Documents owned by user u1:
Research Notes -> {'user_id': 'u1', 'name': 'Kostas', 'email': 'kostas@example.com'}


b) Get a document by ID

In [21]:
print("\nDocument with ID 'doc_1':")
print(documents_col.find_one({"_id":"doc_1"}))


Document with ID 'doc_1':
{'_id': 'doc_1', 'title': 'Research Notes', 'created_at': datetime.datetime(2025, 11, 8, 12, 0), 'owner': {'user_id': 'u1', 'name': 'Kostas', 'email': 'kostas@example.com'}, 'collaborators': [{'user_id': 'u2', 'name': 'Elena', 'role': 'editor'}, {'user_id': 'u3', 'name': 'George', 'role': 'viewer'}], 'edits': [{'edit_id': 'e1', 'user_id': 'u2', 'timestamp': datetime.datetime(2025, 11, 8, 13, 15), 'content_added': 'Introduction paragraph added'}, {'edit_id': 'e2', 'user_id': 'u3', 'timestamp': datetime.datetime(2025, 11, 8, 14, 0), 'content_added': 'Added references section'}]}


c) Add a collaborator

In [24]:
documents_col.update_one(
    {"_id": "doc_2"},
    {"$push": {"collaborators": {"user_id": "u3", "name": "George", "role": "viewer"}}}
)
print("\nAdded collaborator George to doc_2.")



Added collaborator George to doc_2.


d) Remove a collaborator

In [29]:
documents_col.update_one(
    {"_id": "doc_2"},
    {"$pull": {"collaborators":{"user_id": "u3"}}}
)
print("Removed collaborator George from doc_2.")

Removed collaborator George from doc_2.


e) Insert a new document edit

In [32]:
new_edit = { "edit_id": "e3", 
            "user_id": "u1",
            "timestamp": datetime.now(),
            "content_added": "Added project summary section."
           }
documents_col.update_one(
    {"_id": "doc_1"},
    {"$push": {"edits": new_edit}}
)
print("\nInserted new edit into doc_1.")



Inserted new edit into doc_1.


f) Get all edits of a document

In [35]:
doc = documents_col.find_one({"_id": "doc_1"}, {"edits": 1, "_id": 0})
print("\nAll edits for doc_1:")
for edit in doc["edits"]:
    print(edit)


All edits for doc_1:
{'edit_id': 'e1', 'user_id': 'u2', 'timestamp': datetime.datetime(2025, 11, 8, 13, 15), 'content_added': 'Introduction paragraph added'}
{'edit_id': 'e2', 'user_id': 'u3', 'timestamp': datetime.datetime(2025, 11, 8, 14, 0), 'content_added': 'Added references section'}
{'edit_id': 'e3', 'user_id': 'u1', 'timestamp': datetime.datetime(2025, 11, 9, 16, 31, 22, 985000), 'content_added': 'Added project summary section.'}


g) Get a specific edit (edit_id = e2)

In [38]:
doc = documents_col.find_one(
    {"_id": "doc_1", "edits.edit_id": "e2"},
    {"edits.$":1}
)
print("\nSpecific edit e2:")
print(doc["edits"][0])

print("\nAll operations completed successfully!")


Specific edit e2:
{'edit_id': 'e2', 'user_id': 'u3', 'timestamp': datetime.datetime(2025, 11, 8, 14, 0), 'content_added': 'Added references section'}

All operations completed successfully!
